📊 Intelligent Recommendation System - Analysis & Visualization
 
 Notebook này giúp bạn:
 1. Explore dữ liệu TikDataset
 2. Visualize kết quả preprocessing
 3. Analyze model performance
 4. Experiment với recommendations

## 1. Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from tqdm import tqdm

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Check data files
print("Checking data files...")
files = {
    "Item Features": "data/processed/item_features.csv",
    "Ranking Features": "data/processed/ranking_features.csv",
    "Interactions": "data/processed/interactions.csv"
}

for name, path in files.items():
    exists = "✓" if Path(path).exists() else "✗"
    print(f"{exists} {name}: {path}")

## 2. Data Exploration

In [ ]:
# Load data
item_df = pd.read_csv("data/processed/item_features.csv")
ranking_df = pd.read_csv("data/processed/ranking_features.csv")

# Basic info
print("="*60)
print("DATASET OVERVIEW")
print("="*60)
print(f"\nItems: {len(item_df):,}")
print(f"Ranking samples: {len(ranking_df):,}")
print(f"\nCategories: {item_df['category'].nunique():,}")
print(f"Brands: {item_df['brand_id'].nunique():,}")

### 2.1. Category Distribution

In [ ]:
# Top categories
top_categories = item_df['category'].value_counts().head(20)

plt.figure(figsize=(14, 6))
top_categories.plot(kind='barh', color='steelblue')
plt.title('Top 20 Categories by Product Count', fontsize=14, fontweight='bold')
plt.xlabel('Number of Products')
plt.ylabel('Category')
plt.tight_layout()
plt.show()

print("\nTop 10 Categories:")
print(top_categories.head(10))

In [ ]:
# Category distribution pie chart
top_10_cats = item_df['category'].value_counts().head(10)
other_count = len(item_df) - top_10_cats.sum()

# Add "Others"
plot_data = pd.concat([top_10_cats, pd.Series({'Others': other_count})])

plt.figure(figsize=(10, 10))
plt.pie(plot_data, labels=plot_data.index, autopct='%1.1f%%', startangle=90)
plt.title('Product Distribution by Category', fontsize=14, fontweight='bold')
plt.axis('equal')
plt.show()

### 2.2. Price Analysis

In [ ]:
# Price distribution
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
# Remove outliers for better visualization
price_filtered = ranking_df[ranking_df['price'] < ranking_df['price'].quantile(0.95)]
plt.hist(price_filtered['price'], bins=50, color='skyblue', edgecolor='black')
plt.xlabel('Price (VNĐ)')
plt.ylabel('Frequency')
plt.title('Price Distribution (95th percentile)', fontweight='bold')
plt.ticklabel_format(style='plain', axis='x')

plt.subplot(1, 2, 2)
# Log scale
plt.hist(np.log10(ranking_df['price'].replace(0, 1)), bins=50, color='salmon', edgecolor='black')
plt.xlabel('Log10(Price)')
plt.ylabel('Frequency')
plt.title('Price Distribution (Log Scale)', fontweight='bold')

plt.tight_layout()
plt.show()

# Price statistics
print("\nPrice Statistics:")
print(ranking_df['price'].describe())
print(f"\nNumber of unique prices: {ranking_df['price'].nunique():,}")

In [ ]:
# Price by category (top 10)
top_cats = item_df['category'].value_counts().head(10).index
price_by_cat = ranking_df.merge(
    item_df[['product_id', 'category']], 
    on='product_id'
)[['category', 'price']]

price_by_cat = price_by_cat[price_by_cat['category'].isin(top_cats)]

plt.figure(figsize=(14, 6))
sns.boxplot(data=price_by_cat, x='category', y='price', palette='Set2')
plt.xticks(rotation=45, ha='right')
plt.ylabel('Price (VNĐ)')
plt.title('Price Distribution by Top Categories', fontsize=14, fontweight='bold')
plt.ticklabel_format(style='plain', axis='y')
plt.tight_layout()
plt.show()

### 2.3. Rating Analysis

In [ ]:
# Rating distribution
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
rating_counts = ranking_df['rating_average'].value_counts().sort_index()
plt.bar(rating_counts.index, rating_counts.values, color='gold', edgecolor='black')
plt.xlabel('Rating')
plt.ylabel('Number of Products')
plt.title('Rating Distribution', fontweight='bold')
plt.xticks([0, 1, 2, 3, 4, 5])

plt.subplot(1, 2, 2)
# Rating vs Review Count
sample = ranking_df.sample(min(5000, len(ranking_df)))
plt.scatter(sample['review_count'], sample['rating_average'], 
           alpha=0.3, s=10, color='purple')
plt.xlabel('Review Count')
plt.ylabel('Rating Average')
plt.title('Rating vs Review Count', fontweight='bold')
plt.xscale('log')

plt.tight_layout()
plt.show()

# Rating statistics
print("\nRating Statistics:")
print(ranking_df['rating_average'].describe())

### 2.4. Discount Analysis

In [ ]:
# Discount rate distribution
has_discount = ranking_df[ranking_df['discount_rate'] > 0]

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.hist(has_discount['discount_rate'], bins=30, color='green', alpha=0.7, edgecolor='black')
plt.xlabel('Discount Rate (%)')
plt.ylabel('Frequency')
plt.title(f'Discount Distribution ({len(has_discount):,} products)', fontweight='bold')

plt.subplot(1, 2, 2)
# Discount ranges
discount_ranges = pd.cut(
    ranking_df['discount_rate'], 
    bins=[0, 10, 20, 30, 50, 100],
    labels=['0-10%', '10-20%', '20-30%', '30-50%', '50%+']
)
discount_counts = discount_ranges.value_counts().sort_index()
plt.pie(discount_counts, labels=discount_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Discount Range Distribution', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nProducts with discount: {len(has_discount):,} ({len(has_discount)/len(ranking_df)*100:.1f}%)")
print(f"Average discount: {has_discount['discount_rate'].mean():.1f}%")

### 2.5. Badge Analysis

In [ ]:
# Badge counts
badges = {
    'Authentic': ranking_df['is_authentic'].sum(),
    'FreeShip': ranking_df['is_freeship'].sum(),
    'Return Policy': ranking_df['has_return_policy'].sum(),
    'Available': ranking_df['is_available'].sum()
}

plt.figure(figsize=(10, 6))
bars = plt.bar(badges.keys(), badges.values(), color=['gold', 'skyblue', 'lightgreen', 'salmon'])
plt.ylabel('Number of Products')
plt.title('Badge Distribution', fontsize=14, fontweight='bold')

# Add percentages on bars
for i, (badge, count) in enumerate(badges.items()):
    pct = count / len(ranking_df) * 100
    plt.text(i, count, f'{count:,}\n({pct:.1f}%)', 
            ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

### 2.6. Text Length Analysis

In [ ]:
# Text content length
item_df['text_length'] = item_df['text_content'].str.len()

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.hist(item_df['text_length'], bins=50, color='teal', alpha=0.7, edgecolor='black')
plt.xlabel('Text Length (characters)')
plt.ylabel('Frequency')
plt.title('Product Description Length Distribution', fontweight='bold')

plt.subplot(1, 2, 2)
# Box plot by category (top 10)
text_by_cat = item_df[item_df['category'].isin(top_cats)][['category', 'text_length']]
sns.boxplot(data=text_by_cat, x='category', y='text_length', palette='Set3')
plt.xticks(rotation=45, ha='right')
plt.ylabel('Text Length')
plt.title('Description Length by Category', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nText Length Statistics:")
print(item_df['text_length'].describe())

## 3. Model Performance Analysis

In [ ]:
# Load training logs (if available)
try:
    import json
    
    # Assuming you saved training logs
    with open('logs/training_history.json', 'r') as f:
        history = json.load(f)
    
    # Plot Two-Tower training
    if 'two_tower' in history:
        tt_history = history['two_tower']
        
        plt.figure(figsize=(14, 5))
        
        plt.subplot(1, 2, 1)
        plt.plot(tt_history['train_loss'], label='Train Loss', linewidth=2)
        plt.plot(tt_history['val_loss'], label='Val Loss', linewidth=2)
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Two-Tower Model - Training Loss', fontweight='bold')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.subplot(1, 2, 2)
        plt.plot(tt_history['recall_at_10'], label='Recall@10', linewidth=2)
        plt.plot(tt_history['recall_at_50'], label='Recall@50', linewidth=2)
        plt.xlabel('Epoch')
        plt.ylabel('Recall')
        plt.title('Two-Tower Model - Recall Metrics', fontweight='bold')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    # Plot MMoE training
    if 'mmoe' in history:
        mmoe_history = history['mmoe']
        
        plt.figure(figsize=(14, 5))
        
        plt.subplot(1, 2, 1)
        plt.plot(mmoe_history['train_loss'], label='Train Loss', linewidth=2)
        plt.plot(mmoe_history['val_loss'], label='Val Loss', linewidth=2)
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('MMoE Model - Training Loss', fontweight='bold')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.subplot(1, 2, 2)
        tasks = ['Purchase', 'Quality', 'Price']
        for task in tasks:
            plt.plot(mmoe_history[f'auc_{task.lower()}'], label=f'{task} AUC', linewidth=2)
        plt.xlabel('Epoch')
        plt.ylabel('AUC')
        plt.title('MMoE Model - AUC per Task', fontweight='bold')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
except FileNotFoundError:
    print("Training logs not found. Run training first to generate logs.")

## 4. Recommendation Analysis

In [ ]:
# Load recommendation results (if available)
try:
    results_dir = Path("results")
    result_files = list(results_dir.glob("recommendations_user_*.csv"))
    
    if result_files:
        print(f"Found {len(result_files)} recommendation results")
        
        # Load one example
        sample_result = pd.read_csv(result_files[0])
        
        print("\nSample Recommendations:")
        print(sample_result[['product_id', 'category', 'score', 'price', 'rating_average']].head(10))
        
        # Score distribution
        plt.figure(figsize=(10, 5))
        plt.hist(sample_result['score'], bins=30, color='purple', alpha=0.7, edgecolor='black')
        plt.xlabel('Recommendation Score')
        plt.ylabel('Frequency')
        plt.title('Score Distribution in Recommendations', fontweight='bold')
        plt.show()
        
    else:
        print("No recommendation results found. Run demo_inference.py first.")
        
except Exception as e:
    print(f"Could not load results: {e}")

## 5. Interactive Recommendation Demo

In [ ]:
# Mini inference demo (requires models)
try:
    from demo_inference import RecommendationEngine
    
    # Initialize engine
    print("Initializing Recommendation Engine...")
    engine = RecommendationEngine()
    
    # Test recommendation
    user_id = 12345
    recommendations = engine.recommend(user_id, top_n=10)
    
    print(f"\nTop 10 Recommendations for User {user_id}:")
    print(recommendations[['product_id', 'category', 'score', 'price']].to_string())
    
    # Visualize categories in recommendations
    cat_counts = recommendations['category'].value_counts()
    
    plt.figure(figsize=(10, 6))
    plt.pie(cat_counts, labels=cat_counts.index, autopct='%1.1f%%', startangle=90)
    plt.title(f'Category Distribution in Recommendations for User {user_id}', fontweight='bold')
    plt.show()
    
except Exception as e:
    print(f"Could not run inference demo: {e}")
    print("Make sure models are trained first.")

## 6. Export Analysis Report

In [ ]:
# Generate summary report
report = f"""
# RECOMMENDATION SYSTEM ANALYSIS REPORT

## Dataset Statistics
- Total Items: {len(item_df):,}
- Total Categories: {item_df['category'].nunique():,}
- Total Brands: {item_df['brand_id'].nunique():,}

## Price Analysis
- Average Price: {ranking_df['price'].mean():,.0f} VNĐ
- Median Price: {ranking_df['price'].median():,.0f} VNĐ
- Price Range: {ranking_df['price'].min():,.0f} - {ranking_df['price'].max():,.0f} VNĐ

## Rating Analysis
- Average Rating: {ranking_df['rating_average'].mean():.2f} / 5.0
- Products with Reviews: {(ranking_df['review_count'] > 0).sum():,} ({(ranking_df['review_count'] > 0).mean()*100:.1f}%)

## Discount Analysis
- Products with Discount: {(ranking_df['discount_rate'] > 0).sum():,} ({(ranking_df['discount_rate'] > 0).mean()*100:.1f}%)
- Average Discount: {ranking_df[ranking_df['discount_rate'] > 0]['discount_rate'].mean():.1f}%

## Badge Statistics
- Authentic: {ranking_df['is_authentic'].sum():,} ({ranking_df['is_authentic'].mean()*100:.1f}%)
- FreeShip: {ranking_df['is_freeship'].sum():,} ({ranking_df['is_freeship'].mean()*100:.1f}%)
- Return Policy: {ranking_df['has_return_policy'].sum():,} ({ranking_df['has_return_policy'].mean()*100:.1f}%)

Generated: {pd.Timestamp.now()}
"""

# Save report
with open('results/analysis_report.md', 'w', encoding='utf-8') as f:
    f.write(report)

print("✓ Analysis report saved to results/analysis_report.md")
print(report)